In [1]:
# Ingest list of suppliers
import numpy as np
import pandas as pd

suppliers = pd.read_csv("isss608-g3-project\data\GovernmentProcurementviaGeBIZ.csv")
print(suppliers.info())

print(suppliers["supplier_name"].nunique())

supplier_names = suppliers["supplier_name"].unique().tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18638 entries, 0 to 18637
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tender_no             18638 non-null  object 
 1   tender_description    18638 non-null  object 
 2   agency                18638 non-null  object 
 3   award_date            18638 non-null  object 
 4   tender_detail_status  18638 non-null  object 
 5   supplier_name         18638 non-null  object 
 6   awarded_amt           18638 non-null  float64
dtypes: float64(1), object(6)
memory usage: 1019.4+ KB
None
6230


In [3]:
# Scrape data from GeBIZ Database with selenium
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

gebiz_site_url = "https://www.gebiz.gov.sg/"

options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)
wait = WebDriverWait(driver, 1)

driver.get(gebiz_site_url)
supplier_directory = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[1]/div[4]/div[3]/div[3]/span/a')))
supplier_directory.click()

extracted_data = []
no_results = []

for name in supplier_names:
    try:
        # Insert names and search
        input_field = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/form[2]/div[6]/div/div/div[2]/div/div/div/div/div[2]/div/div[1]/div/div[2]/div/div/div/div/div/input[1]')))        
        input_field.click()
        input_field.clear()
        input_field.send_keys(name)

        search_field = driver.find_element(By.XPATH, '//*[@id="contentForm:search"]')
        search_field.click()

        # Get the first result, if there is one
        first_result = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/form[2]/div[6]/div/div/div[3]/div/div/div/div/div[3]/div/div[2]/div/div/div/div[3]/div/div/div/div/div/span/a')))
        first_result.click()

        desc_text = ""
        country_text = ""
        postal_code = ""
        financial_grade = ""
        # Extract data
        ## company description
        desc = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/form[2]/div[6]/div/div/div[1]/div/div/div/div/div[1]/div/div/div/div/div[2]/div/div/div/div[2]/div')))
        if desc:
            desc_text = desc.text
        ## country
        country = driver.find_element(By.XPATH, '/html/body/div/form[2]/div[6]/div/div/div[2]/div/div/div/div/div[1]/div/div/div/div/div[2]/div/div/div/div[2]/div')
        if country:
            country_text = country.text
        ## postal code
        postal = driver.find_element(By.XPATH, '/html/body/div/form[2]/div[6]/div/div/div[2]/div/div/div/div/div[1]/div/div/div/div/div[9]/div/div/div/div[2]/div')
        if postal:
            postal_code = postal.text
        ## business categories
        main_cat = []
        sub_cat = []
        sub_categories = driver.find_elements(By.XPATH, "//div[contains(@class, 'LEAF')]")
        for sub in sub_categories:
            try:
                text = sub.find_element(By.XPATH, ".//span").text.strip()
                sub_cat.append(text)
            except:
                pass
        main_categories = driver.find_elements(By.CLASS_NAME, "formDataTree_ROW")
        for main in main_categories:
            try:
                text = main.find_element(By.XPATH, ".//span").text.strip()
                if text and text.lower() != "category" and text not in sub_cat:
                    main_cat.append(text)
            except:
                pass
        ## supply heads
        supply_heads = []
        rows = driver.find_elements(By.XPATH, "//tr[contains(@id, 'contentForm:j_idt189_row_')]")
        for row in rows:
            try:
                supply_head_element = row.find_element(By.XPATH, "./td[2]/span") #second td
                supply_heads.append(supply_head_element.text.strip())
            except:
                pass
        ## financial grade
        fg = driver.find_element(By.XPATH, '/html/body/div/form[2]/div[6]/div/div/div[4]/div/div/div/div/div[1]/div/div/div/div/div[2]/div/table/tbody/tr[1]/td[3]/span')
        if fg:
            financial_grade = fg.text
    
        extracted_data.append({
            'supplier_name': name,
            'company_desc': desc_text,
            'country': country_text,
            'postal_code': postal_code,
            'main_business_category': "; ".join(main_cat),
            'sub_business_category': "; ".join(sub_cat),
            'supply_heads': "; ".join(supply_heads),
            'financial_grade': financial_grade
        })

        back_button = driver.find_element(By.XPATH, '/html/body/div/form[2]/div[5]/div/div/div/div/div/div/div[5]/div/div/div/input')
        back_button.click()
    
    except Exception as e:
        print(f'No result for {name}')
        no_results.append({
            'supplier_name': name
        })
        continue

# close the drivers
driver.quit()

extracted_df = pd.DataFrame(extracted_data)
no_results_df = pd.DataFrame(no_results)

The geckodriver version (0.35.0) detected in PATH at C:\Users\yjuny\AppData\Local\Microsoft\WindowsApps\geckodriver.exe might not be compatible with the detected firefox version (136.0.1.595); currently, geckodriver 0.36.0 is recommended for firefox 136.*, so it is advised to delete the driver in PATH and retry


No result for Unknown
No result for INSIGHTMATRIX
No result for ACTIVEO SINGAPORE PTE. LTD.
No result for Checkbox Technology Pty Ltd
No result for INTERSECT PTE. LTD.
No result for OPUS 2 INTERNATIONAL SINGAPORE PTE. LTD.
No result for CIVIC LEGAL LLC
No result for LEGAL SOLUTIONS LLC
No result for TEO KENG SIANG LLC
No result for NETPLUZ TELECOMS PTE. LTD.
No result for CUBIC SPACE PRIVATE LIMITED
No result for GOLDCASTLE INTERNATIONAL PTE LTD
No result for TMA EVENTS PTE. LTD.
No result for TACTICS SYSTEM ENGINEERING PTE LTD
No result for ADVANCED CONSTRUCTION & ENGINEERING PTE LTD
No result for SHIN KHAI CONSTRUCTION PTE. LTD.
No result for SOLARGY PTE. LTD.
No result for TERRA SG PTE. LTD.
No result for NOVACITYNETS PTE LTD
No result for CHL CONSTRUCTION PTE LTD
No result for SINTRUST CONSTRUCTION PTE. LTD.
No result for EXMILE SOLUTIONS LIMITED
No result for ADVOCATUS LAW LLP
No result for HOLBORN LAW LLC
No result for CSD INDUSTRIES PTE LTD
No result for YU YEO CONSTRUCTION PTE 

In [4]:
extracted_df.to_csv("isss608-g3-project\data\supplier_details.csv", index=False)
no_results_df.to_csv("isss608-g3-project\data\supplier_no_results.csv", index=False)
extracted_df

,supplier_name,company_desc,country,postal_code,main_business_category,sub_business_category,supply_heads,financial_grade
0,AZAAS PTE. LTD.,aZaaS provides IT solutions with a specializat...,SINGAPORE,068912,IT & Telecommunication; Others,Computer Accessories; Desktop Computers; IT Se...,"EPU/CMP/10 - Computer Related Hardware, Softwa...","S9 $30,000,000 (EPU S9)"
1,ACCENTURE SG SERVICES PTE. LTD.,Accenture is a leading global professional ser...,SINGAPORE,179101,IT & Telecommunication; Others; Services,IT Services & Software Development; System Con...,"EPU/CMP/10 - Computer Related Hardware, Softwa...","S10 >$30,000,000 (EPU S10)"
2,TECH MAHINDRA LIMITED (SINGAPORE BRANCH),Tech Mahindra offers innovative and customer-c...,SINGAPORE,486025,IT & Telecommunication; Services,Computer Accessories; Desktop Computers; IT Se...,EPU/CNE/10 - Communication and Navigation Syst...,"S10 >$30,000,000 (EPU S10)"
3,TDCX (SG) PTE. LTD.,,SINGAPORE,469004,Miscellaneous; Others; Services,"Call Centre; Data Entry, Supply of Manpower Se...","EPU/CMP/10 - Computer Related Hardware, Softwa...","S10 >$30,000,000 (EPU S10)"
4,DELOITTE & TOUCHE ENTERPRISE RISK SERVICES PTE...,,SINGAPORE,068809,IT & Telecommunication; Services,IT Services & Software Development; Softwares ...,"EPU/CMP/10 - Computer Related Hardware, Softwa...","S10 >$30,000,000 (EPU S10)"
...,...,...,...,...,...,...,...,...
3047,CARROTS CONSULTING PTE LTD,CarroCarrots Consulting is an executive compen...,SINGAPORE,068914,Services,Professional Services,EPU/SER/34 - Service (Consultant); EPU/SER/30 ...,"S8 $10,000,000 (EPU S8)"
3048,CHARTERHOUSE PTE. LTD.,Charterhouse is a boutique executive search fi...,SINGAPORE,049908,Miscellaneous; Others,Executive Search,"EPU/SER/19 - Service (Data Entry, Supply of Ma...","S8 $10,000,000 (EPU S8)"
3049,OPUS IT SERVICES PTE LTD,,SINGAPORE,417938,IT & Telecommunication; Services,Computer Accessories; Desktop Computers; IT Se...,"EPU/AVP/10 - Audio Visual,Photographic & Optic...","S10 >$30,000,000 (EPU S10)"
3050,FABRISTEEL PRIVATE LIMITED,,SINGAPORE,628717,Administration & Training; Construction; Other...,Educational Supplies; Interior Decoration; Ren...,EPU/HEQ/10 - Domestic Equipment & Supplies; EP...,"S10 >$30,000,000 (EPU S10)"
